# Import libraries

In [ ]:
import geotrellis.raster._
import geotrellis.raster.vectorize._
import geotrellis.raster.vectorize
import geotrellis.raster.io.geotiff._
import geotrellis.raster.render._
import geotrellis.raster.io.geotiff.GeoTiff
import geotrellis.raster.resample._

import geotrellis.spark._
import geotrellis.spark.io._
import geotrellis.spark.io.RasterReader
import geotrellis.spark.io.hadoop._
import geotrellis.spark.io.hadoop
import geotrellis.spark.tiling._
import geotrellis.spark.tiling.FloatingLayoutScheme

import geotrellis.vector._

import org.apache.spark.SparkContext
import org.apache.spark.rdd._
import org.apache.spark.rdd.RDD
import org.apache.spark.HashPartitioner
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}

import java.net.URI
import scala.math.BigDecimal.RoundingMode
import org.apache.hadoop.fs.Path

# Important variables

In [ ]:
val rr = implicitly[RasterReader[HadoopGeoTiffRDD.Options, (ProjectedExtent, Tile)]]
implicit val sparkContext = sc

rr: geotrellis.spark.io.RasterReader[geotrellis.spark.io.hadoop.HadoopGeoTiffRDD.Options,(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = geotrellis.spark.io.RasterReader$$anon$1@61147ded
sparkContext: org.apache.spark.SparkContext = org.apache.spark.SparkContext@298f7509


# Parameters

In [ ]:
val HdfsUrl = "hdfs://hupi-factory-02-01-01-01/"
val dataRepo = "user/factory02/thailand_workshop/data_multiBands/"
val saveRepo = "user/factory02/thailand_workshop/kmeans/"
val landsatName = "LC08_L1TP_125052_20171231_20180103_01_T1"
val k = 9
// list of colors for PNG (17 colors for 17 models)
val allColors = "0:669999ff;1:8cd98cff;2:ffff00ff;3:000099ff;4:737373ff;5:006666ff;6:4dffffff;7:ff9f80ff;8:00ccffff;9:339966ff;10:00ffccff;11:6600ffff;12:cc3300ff;13:66ffccff;14:000000ff;15:ff66ffff;16:0066ccff;17:ff8000ff;18:3399ffff;19:003366ff"

HdfsUrl: String = hdfs://hupi-factory-02-01-01-01/
dataRepo: String = user/factory02/thailand_workshop/data_multiBands/
saveRepo: String = user/factory02/thailand_workshop/kmeans/
landsatName: String = LC08_L1TP_125052_20171231_20180103_01_T1
k: Int = 9
allColors: String = 0:669999ff;1:8cd98cff;2:ffff00ff;3:000099ff;4:737373ff;5:006666ff;6:4dffffff;7:ff9f80ff;8:00ccffff;9:339966ff;10:00ffccff;11:6600ffff;12:cc3300ff;13:66ffccff;14:000000ff;15:ff66ffff;16:0066ccff;17:ff8000ff;18:3399ffff;19:003366ff


In [ ]:
// List of k colors
val listColors = allColors.split(";").take(k).mkString(";")

listColors: String = 0:669999ff;1:8cd98cff;2:ffff00ff;3:000099ff;4:737373ff;5:006666ff;6:4dffffff;7:ff9f80ff;8:00ccffff


In [ ]:
val saveAddress = HdfsUrl + saveRepo + landsatName + "/" + k
val savePath_illustration = new Path(saveAddress + ".png")

saveAddress: String = hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/kmeans/LC08_L1TP_125052_20171231_20180103_01_T1/9
savePath_illustration: org.apache.hadoop.fs.Path = hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/kmeans/LC08_L1TP_125052_20171231_20180103_01_T1/9.png


# To avoid duplicates in HDFS

In [ ]:
val conf = sc.hadoopConfiguration 
val fs = org.apache.hadoop.fs.FileSystem.get(new java.net.URI(HdfsUrl), conf)

conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
fs: org.apache.hadoop.fs.FileSystem = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-717805646_122, ugi=root (auth:SIMPLE)]]


In [ ]:
fs.delete(savePath_illustration,true)

res8: Boolean = false


false

# Load multiBandgeoTiff from HDFS

In [ ]:
val options =
HadoopGeoTiffRDD.Options(
  numPartitions = Some(100)
)

In [ ]:
// We get multi bands from HDFS (except band 8) 
val sourceTiles = sc.hadoopMultibandGeoTiffRDD(HdfsUrl + dataRepo + landsatName + ".tif").repartition(100)

sourceTiles: org.apache.spark.rdd.RDD[(geotrellis.vector.ProjectedExtent, geotrellis.raster.MultibandTile)] = MapPartitionsRDD[7] at repartition at <console>:120


In [ ]:
// We convert multi tiles into one vector of features in RDD
val rddArrayOfTiles = sourceTiles.map (l => (l._2.band(0).convert(DoubleConstantNoDataCellType).toArrayDouble(), 
                       l._2.band(1).convert(DoubleConstantNoDataCellType).toArrayDouble(), 
                       l._2.band(2).convert(DoubleConstantNoDataCellType).toArrayDouble(),
                       l._2.band(3).convert(DoubleConstantNoDataCellType).toArrayDouble(), 
                       l._2.band(4).convert(DoubleConstantNoDataCellType).toArrayDouble(),
                       l._2.band(5).convert(DoubleConstantNoDataCellType).toArrayDouble(), 
                       l._2.band(6).convert(DoubleConstantNoDataCellType).toArrayDouble(),
                       l._2.band(7).convert(DoubleConstantNoDataCellType).toArrayDouble(),
                       l._2.band(8).convert(DoubleConstantNoDataCellType).toArrayDouble(), 
                       l._2.band(9).convert(DoubleConstantNoDataCellType).toArrayDouble()))
.map(l => l._1.zip(l._2).zip(l._3).zip(l._4).zip(l._5).zip(l._6).zip(l._7).zip(l._8).zip(l._9).zip(l._10))
.map(l => l.map(k => Vectors.dense(k._1._1._1._1._1._1._1._1._1, k._1._1._1._1._1._1._1._1._2, 
                                   k._1._1._1._1._1._1._1._2, k._1._1._1._1._1._1._2,
                                  k._1._1._1._1._1._2, k._1._1._1._1._2, k._1._1._1._2,
                                   k._1._1._2, k._1._2, k._2)))

rddArrayOfTiles: org.apache.spark.rdd.RDD[Array[org.apache.spark.mllib.linalg.Vector]] = MapPartitionsRDD[10] at map at <console>:134


In [ ]:
val input = rddArrayOfTiles.flatMap(l => l)

input: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.Vector] = MapPartitionsRDD[11] at flatMap at <console>:124


# Load model to HDFS

In [ ]:
// To load the model (for later use)
val clusters = KMeansModel.load(sc, saveAddress)

clusters: org.apache.spark.mllib.clustering.KMeansModel = org.apache.spark.mllib.clustering.KMeansModel@4d023088


# To find the clusters of the input

In [ ]:
val results = clusters.predict(input)

results: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[21] at map at KMeansModel.scala:69


In [ ]:
// number of element for each cluster
results.map(l => (l, 1)).reduceByKey(_ + _).take(10)

res15: Array[(Int, Int)] = Array((0,11383641), (1,19517793), (2,3990193), (3,2643728), (4,892626), (5,2579935), (6,5605074), (7,6676567), (8,7658923))


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
/*
val res = input.zip(results)

// For cluster 0, we have
res.filter(l => l._2 == 0).take(100)
*/

# Visualization results

From the multibandGeoTiff in HDFS, we will find the clusters of each pixel and create a PNG of k colors corresponding to number cluster.

### 1/ To do that, firstly, we need to find the ProjectedExtent of the results

In [ ]:
// We zip results of clusters with index 
val resultsWithIndex = results.zipWithIndex

resultsWithIndex: org.apache.spark.rdd.RDD[(Int, Long)] = ZippedWithIndexRDD[24] at zipWithIndex at <console>:134


In [ ]:
// From sourceTiles, we need to get the number of columns and rows of each tile
val nbColsAndRows = sourceTiles.map(l => (l._2.cols, l._2.rows)).zipWithIndex.map(l => (l._2, l._1))

nbColsAndRows: org.apache.spark.rdd.RDD[(Long, (Int, Int))] = MapPartitionsRDD[27] at map at <console>:122


In [ ]:
// We compute lengths (number of pixels) of each tile 
val lengthTiles = nbColsAndRows.map(l => l._2._1 * l._2._2).collect()

lengthTiles: Array[Int] = Array(65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65536, 65...

In [ ]:
// Then we should find the key (index of tile) for each pixel

// We compute the cumulative sum of the length of tiles
var old = 0
val cumSum = lengthTiles.map {
  case (l) => 
  {
    old = old + l
    old
  }
}

// We got the key by doing the logic below 
val resultsWithKey = resultsWithIndex.map {
  case (value, index) => 
  {
    var ind = 0
    while(index >= cumSum(ind)) {
      ind = ind + 1
    } 
    (ind, value)
  }
}

old: Int = 60948480
cumSum: Array[Int] = Array(65536, 131072, 196608, 262144, 327680, 393216, 458752, 524288, 589824, 655360, 720896, 786432, 851968, 917504, 983040, 1048576, 1114112, 1179648, 1245184, 1310720, 1376256, 1441792, 1507328, 1572864, 1638400, 1703936, 1769472, 1835008, 1900544, 1966080, 2031616, 2097152, 2162688, 2228224, 2293760, 2359296, 2424832, 2490368, 2555904, 2621440, 2686976, 2752512, 2818048, 2883584, 2949120, 3014656, 3080192, 3145728, 3211264, 3276800, 3342336, 3407872, 3473408, 3538944, 3604480, 3670016, 3735552, 3801088, 3866624, 3932160, 3997696, 4063232, 4128768, 4194304, 4259840, 4325376, 4390912, 4456448, 4521984, 4587520, 4653056, 4718592, 4784128, 4849664, 4915200, 4980736, 5046272, 5111808, 5177344, 5242880, 5308416, 5373952, 5439488, 5505024, 5570560, 5...

In [ ]:
// We group by key 
val resultsGrouped = resultsWithKey.map(l => (l._1.toLong, Array(l._2))).reduceByKey(_ ++ _)

resultsGrouped: org.apache.spark.rdd.RDD[(Long, Array[Int])] = ShuffledRDD[31] at reduceByKey at <console>:144


In [ ]:
// Now we got the same count as sourceTiles.count()
// resultsGrouped.count()

In [ ]:
// Then, we zip with index for RDD[ProjectedExtent]
val peWithIndex = sourceTiles.map(l => l._1).zipWithIndex.map(l => (l._2, l._1))

peWithIndex: org.apache.spark.rdd.RDD[(Long, geotrellis.vector.ProjectedExtent)] = MapPartitionsRDD[34] at map at <console>:122


In [ ]:
// RDD of ProjectExtented with index of cluster and nb_cols and nb_rows
val peWithClusterWithColsAndRows = peWithIndex.join(resultsGrouped).join(nbColsAndRows)

peWithClusterWithColsAndRows: org.apache.spark.rdd.RDD[(Long, ((geotrellis.vector.ProjectedExtent, Array[Int]), (Int, Int)))] = MapPartitionsRDD[40] at join at <console>:148


In [ ]:
// Convert to IntArrayTile
val kmeansTile = peWithClusterWithColsAndRows.map(l => (l._2._1._1, IntArrayTile(l._2._1._2, l._2._2._1, l._2._2._2).tile))

kmeansTile: org.apache.spark.rdd.RDD[(geotrellis.vector.ProjectedExtent, geotrellis.raster.Tile)] = MapPartitionsRDD[41] at map at <console>:150


### Stiching Tiles into a single Geotiff

In [ ]:
// Tile this RDD to a grid layout. This will transform our raster data into a
// common grid format, and merge any overlapping data.
val layoutScheme = FloatingLayoutScheme()

// We gather the metadata that we will be targeting with the tiling here.
// The return also gives us a zoom level, which we ignore.
val (_: Int, metadata: TileLayerMetadata[SpatialKey]) = kmeansTile.collectMetadata[SpatialKey](layoutScheme)

layoutScheme: geotrellis.spark.tiling.FloatingLayoutScheme = geotrellis.spark.tiling.FloatingLayoutScheme@2d5eeec6
metadata: geotrellis.spark.TileLayerMetadata[geotrellis.spark.SpatialKey] = TileLayerMetadata(int32,GridExtent(Extent(528585.0, 1156335.0, 758985.0, 1394415.0),30.0,30.0),Extent(528585.0, 1156335.0, 758985.0, 1394415.0),EPSG:32648,KeyBounds(SpatialKey(0,0),SpatialKey(29,30)))


In [ ]:
// Here we set some options for our tiling.
// For this example, we will set the target partitioner to one
// that has the same number of partitions as our original RDD.
val tilerOptions =
  Tiler.Options(
    resampleMethod = Bilinear,
    partitioner = new HashPartitioner(kmeansTile.partitions.length)
  )

tilerOptions: geotrellis.spark.tiling.Tiler.Options = Options(Bilinear,Some(org.apache.spark.HashPartitioner@64))


In [ ]:
// Now we tile to an RDD with a SpaceTimeKey.
val tiledRdd = kmeansTile.tileToLayout[SpatialKey](metadata, tilerOptions)

tiledRdd: org.apache.spark.rdd.RDD[(geotrellis.spark.SpatialKey, geotrellis.raster.Tile)] = ShuffledRDD[44] at reduceByKey at TileRDDMerge.scala:48


In [ ]:
// At this point, we want to combine our RDD and our Metadata to get a TileLayerRDD[SpatialKey]
val layerRdd: TileLayerRDD[SpatialKey] = ContextRDD(tiledRdd, metadata)

layerRdd: geotrellis.spark.TileLayerRDD[geotrellis.spark.SpatialKey] = ContextRDD[45] at RDD at ContextRDD.scala:35


In [ ]:
// We stitch all tiles into one Raster
val raster_kmeans = layerRdd.stitch

raster_kmeans: geotrellis.raster.Raster[geotrellis.raster.Tile] = Raster(IntConstantNoDataArrayTile([I@34d6400f,7680,7936),Extent(528585.0, 1156335.0, 758985.0, 1394415.0))


### Convert tile to png and save to HDFS 

In [ ]:
// Get color map from the application.conf settings file.
val colorMap = ColorMap.fromStringDouble(listColors).get

colorMap: geotrellis.raster.render.ColorMap = geotrellis.raster.render.DoubleColorMap@7c22b259


In [ ]:
// Convert tile to png
val kmeans_png = raster_kmeans.tile.renderPng(colorMap)

kmeans_png: geotrellis.raster.render.Png = Png([B@185aa58c)


In [ ]:
// Save png to HDFS
kmeans_png.write(savePath_illustration)

No codec found for hdfs://hupi-factory-02-01-01-01/user/factory02/thailand_workshop/kmeans/LC08_L1TP_125052_20171231_20180103_01_T1/9.png, writing without compression.
